# Backtesting Analysis
This project is to demonstrate how to utilize backtesting to assess stock performance, further increasing the likelihood and filter down to the best stock candidates. 

## What is Backtesting?
Backtesting with Simple Moving Averages (SMAs) in Python is a good step in evaluating the effectiveness of an investment strategy before risking real money in the financial markets. This approach use simulating trades using historical price data to assess how a strategy would have performed in the past. 

By applying these rules to historical price data, key performance metrics can be measured, such as returns, drawdowns, and risk-adjusted ratios. These results provide valuable insights into the strategy's historical profitability and risk profile, helping traders make informed decisions. To summarize, backtesting gives sort of a conclusion to investors where `"if a stock is performing well in the past, it is more likely to perform well in the future.`

This Backtesting Analysis can be done with the use of a library called `Backtesting.py` which simplifies the process by passing the stock listing of choice and its history, the number of cash to invest, and the method of strategy to be used as an argument.

In [1]:
# import libraries
from datetime import datetime
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import random
import warnings
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('exports/filtered_class_a_b.csv')

## Part I: Singular Ticker Testing
Retrieving the stock symbol candidate historical prices (yfinance)
The Backtesting.py library requires a "bring your own data" approach which can be easily retrieved through Yahoo Finance with yfinance API. The historical prices are retrieved which will be used to be analyzed. 

To demonstrate, one of the potential candidates that successfully went through the filtration by its dividend and foundational health (class A or B) will be used for this backtesting. A 1 year period is simulated for this demonstration.

In [3]:
# Example OHLC daily data for Google Inc.
import backtesting
from backtesting.test import GOOG
backtesting.set_bokeh_output(notebook=False)

ticker = yf.Ticker("5248.KL")

C:\Users\Aidan\anaconda3\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [5]:
# Get historical data for the ticker
hist = ticker.history(period="1y")
print(hist)

                               Open      High       Low     Close    Volume  \
Date                                                                          
2022-10-05 00:00:00+08:00  1.733522  1.751867  1.724350  1.751867   2795400   
2022-10-06 00:00:00+08:00  1.742695  1.761039  1.742695  1.761039   1495300   
2022-10-07 00:00:00+08:00  1.761039  1.788555  1.751867  1.779383   2550400   
2022-10-11 00:00:00+08:00  1.779383  1.788555  1.733522  1.733522   2894700   
2022-10-12 00:00:00+08:00  1.742694  1.761039  1.715178  1.742694   1729000   
2022-10-13 00:00:00+08:00  1.761333  1.761333  1.733375  1.752014    675000   
2022-10-14 00:00:00+08:00  1.752014  1.779971  1.752014  1.770652   2849000   
2022-10-17 00:00:00+08:00  1.779971  1.779971  1.752014  1.770652   3699200   
2022-10-18 00:00:00+08:00  1.789290  1.798610  1.761333  1.770652   1430600   
2022-10-19 00:00:00+08:00  1.770652  1.798610  1.770652  1.798610   1049500   
2022-10-20 00:00:00+08:00  1.798610  1.798610  1.752

## Defining Backtesting Strategy with Simple Moving Average

This Python backtesting code implements a simple moving average (SMA) crossover trading strategy using the Backtesting library. In this strategy, two SMAs are defined: a shorter-term SMA (n1) and a longer-term SMA (n2). The key idea is to use these SMAs to identify potential buy and sell signals based on their crossovers.


- When the shorter-term SMA (sma1) crosses above the longer-term SMA (sma2), it triggers a buy signal. Any existing positions (long or short) are closed, and a long position (buy) is initiated.
- Conversely, when sma1 crosses below sma2, it triggers a sell signal. Again, existing positions are closed, and a short position (sell) is initiated.

This strategy aims to capture potential trends in the asset's price movements by leveraging SMA crossovers. Backtesting is then used to evaluate the strategy's historical performance based on these rules, providing insights into its profitability and effectiveness.

In [90]:
import pandas as pd


def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [91]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

## Running the Backtest

In [92]:
from backtesting import Backtest

bt = Backtest(hist, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

Start                     2022-10-05 00:00...
End                       2023-10-05 00:00...
Duration                    365 days 00:00:00
Exposure Time [%]                   62.809917
Equity Final [$]                 10660.898978
Equity Peak [$]                  11448.178949
Return [%]                            6.60899
Buy & Hold Return [%]               12.909115
Return (Ann.) [%]                    6.891294
Volatility (Ann.) [%]               14.689145
Sharpe Ratio                         0.469142
Sortino Ratio                        0.789492
Calmar Ratio                         0.868481
Max. Drawdown [%]                   -7.934885
Avg. Drawdown [%]                   -2.677627
Max. Drawdown Duration      146 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                    7
Win Rate [%]                        57.142857
Best Trade [%]                        3.84227
Worst Trade [%]                     -1.059261
Avg. Trade [%]                    

## Analyzing the Results

- Return [%]: This is the total return on your investment as a percentage. It shows how much your initial capital has grown or shrunk.


- Buy & Hold Return [%]: This is the return you would have achieved if you simply bought and held the asset without any trading. It provides a benchmark for comparison.


- Volatility (Ann.) [%]: This represents the annualized volatility of your strategy. It measures the variation in returns over time. Lower volatility is generally preferred.


- Sharpe Ratio: The Sharpe Ratio measures the risk-adjusted return of your strategy. A higher Sharpe Ratio indicates better risk-adjusted performance. It's a common metric for assessing investment strategies.


- Sortino Ratio: Similar to the Sharpe Ratio, the Sortino Ratio measures risk-adjusted return but focuses on downside risk (volatility of negative returns). A higher Sortino Ratio indicates better risk-adjusted performance, especially in strategies that aim to minimize downside risk.


- Calmar Ratio: The Calmar Ratio measures the risk-adjusted return relative to the maximum drawdown. A higher Calmar Ratio is generally better, as it indicates better returns relative to the risk of significant losses.


- Win Rate [%]: This is the percentage of profitable trades out of the total number of trades. A higher win rate is generally preferred.


- Avg. Trade [%]: This is the average percentage return of all trades in your strategy. It provides a measure of the typical trade performance.


- Avg. Trade Duration: This is the average duration (in days) of all trades.


- Profit Factor: The Profit Factor measures the ratio of gross profit to gross loss. A higher profit factor is generally desirable.


- Expectancy [%]: Expectancy represents the average amount you can expect to win (or lose) per trade.

# Interpreting the Results:

Ideal ranges for backtesting metrics can vary depending on your specific trading strategy, risk tolerance, and investment goals. However, I can provide some general guidelines for what traders often consider ideal or acceptable ranges for key backtesting metrics in a 12-month backtest. Keep in mind that these ranges are not set in stone, and what is considered ideal can differ from one trader to another.

1. **Return [%]:**
   - Ideal Range: Positive returns are typically desirable. Aiming for a return that outperforms a benchmark (e.g., Buy & Hold) is a common goal.

2. **Sharpe Ratio:**
   - Ideal Range: A Sharpe Ratio greater than 1 is often considered good, indicating a positive risk-adjusted return. Higher values are better.

3. **Sortino Ratio:**
   - Ideal Range: A Sortino Ratio greater than 1 is generally seen as positive. It focuses on minimizing downside risk, so a higher Sortino Ratio is preferred.

4. **Calmar Ratio:**
   - Ideal Range: A Calmar Ratio greater than 1 is typically desired. It assesses risk-adjusted return relative to maximum drawdown. Higher values are better.

5. **Max. Drawdown [%]:**
   - Ideal Range: A lower maximum drawdown is better. Ideally, it should be less than 20% for most traders, but this can vary based on risk tolerance.

6. **Volatility (Ann.) [%]:**
   - Ideal Range: Lower volatility is generally preferred. Volatility should be consistent with your risk tolerance and investment goals.

7. **Win Rate [%]:**
   - Ideal Range: A win rate above 50% is typically desirable. A higher win rate indicates more winning trades.

8. **Profit Factor:**
   - Ideal Range: A profit factor greater than 1 is usually considered good. It indicates that the strategy's gross profit outweighs gross losses.

10. **Expectancy [%]:**
    - Ideal Range: A positive expectancy indicates that, on average, each trade contributes positively to the strategy's performance.

Note that these ranges are general guidelines and may vary based on the specific trading strategy and risk tolerance. Some traders may be willing to accept higher drawdowns in exchange for potentially higher returns, while others prioritize lower risk and stability.

Additionally, backtesting is just one part of the evaluation process. Real-world trading involves factors like slippage, transaction costs, and market conditions that can impact performance. It's essential to consider these factors and conduct thorough analysis before implementing a trading strategy in live markets.

In [93]:
from bokeh.plotting import show

# Assuming you have created the 'bt' plot object
bt.plot()



Row(id='8884', ...)

# Part II: Bulk Ticker Testing & Export to CSV

In [85]:
df = pd.read_csv('exports/filtered_class_a_b.csv')

In [86]:

results = []

# Loop through each ticker symbol in the CSV file
for ticker_symbol in df['Ticker']:
    # Construct the ticker object
    ticker = yf.Ticker(ticker_symbol)

    # Get historical data for the ticker
    hist = ticker.history(period="1y")

    # Perform backtesting for the ticker
    bt = Backtest(hist, SmaCross, cash=10_000, commission=.002)
    stats = bt.run()

    # Extract the selected columns from the stats DataFrame
    selected_stats = stats[['Return [%]', 'Buy & Hold Return [%]', 'Return (Ann.) [%]', 
                            'Volatility (Ann.) [%]', 'Avg. Drawdown [%]', 'Win Rate [%]', 
                            'Avg. Trade [%]', 'Profit Factor', 'Expectancy [%]']]

    # Convert selected_stats to a DataFrame with a single row
    selected_stats_df = selected_stats.to_frame().T

    # Reset the index to make it a row
    selected_stats_df = selected_stats_df.reset_index(drop=True)

    # Rename the columns
    selected_stats_df.columns = ['Return [%]', 'Buy & Hold Return [%]', 'Return (Ann.) [%]',
                                'Volatility (Ann.) [%]', 'Avg. Drawdown [%]', 'Win Rate [%]',
                                'Avg. Trade [%]', 'Profit Factor', 'Expectancy [%]']

    # Append the results to the list along with the Ticker symbol
    results.append({
        'Ticker': ticker_symbol,
        **selected_stats_df.iloc[0].to_dict(),  # Use ** to merge dictionaries
    })

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Now, you have the backtesting results for each ticker in the CSV file


In [87]:
results_df

,Ticker,Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],Avg. Drawdown [%],Win Rate [%],Avg. Trade [%],Profit Factor,Expectancy [%]
0,7090.KL,-15.279123,18.194612,-15.797803,16.697847,-8.074911,42.857143,-2.340956,0.364847,-2.191597
1,5248.KL,-16.240919,40.992474,-16.852077,11.813130,-14.308751,25.000000,-4.334240,0.421835,-3.918634
2,2062.KL,-22.433148,12.612158,-23.243117,19.333762,-8.925984,0.000000,-4.954056,0.000000,-4.830451
3,6139.KL,5.137622,12.909115,5.355509,13.812805,-2.860917,66.666667,1.684206,7.699132,1.726085
